# This notebook is to help us test the queries and define the ones to track the alerts 

In [33]:
import pandas as pd
from google.cloud import bigquery
from datetime import datetime,timezone,timedelta, date
from dateutil.relativedelta import relativedelta
BQ_TABLE="rax-architecture-sandbox.rax_gcp_billing.gcp_billing_export_v1_018918_BC80F7_298FEC"

In [5]:
# Thresholds
AMOUNT = 0.9
AMOUNT_CHANGED = 1
PERCENTAGE = 1.01
def parse_cost_changes(rows):
    flagged_items = []
    for item in rows:
        i1 = float(item["prev_day"])
        i2 = float(item["curr_day"])
        if (i1 > AMOUNT or i2 > AMOUNT) or (i1 != 0 and i2 != 0 and ((i2 /i1) >= PERCENTAGE or (i1 /i2) >= PERCENTAGE)) or abs(i2-i1) >= AMOUNT_CHANGED:
            flagged_items.append(item)
    return flagged_items

## This query is to get the daily increase in consumption

In [6]:
client = bigquery.Client()
now = datetime.now(timezone.utc)
prev_utc = now -timedelta(2)
curr_utc = now - timedelta(1)

QUERY = (
    f"""SELECT
    project.name as project_name,
    sku.id as sku_id,
    sku.description as sku_def,
    service.id as service_id,
    service.description as service_def,
    SUM(CASE WHEN EXTRACT(DAY FROM usage_start_time) = {prev_utc.day} THEN cost ELSE 0 END) AS prev_day,
    SUM(CASE WHEN EXTRACT(DAY FROM usage_start_time) = {curr_utc.day} THEN cost ELSE 0 END) AS curr_day,
    FROM `{BQ_TABLE}`
    WHERE project.name = 'rax-datascience' 
    AND (DATE_TRUNC(usage_start_time, DAY) = "{prev_utc.strftime('%Y-%m-%d')}" or DATE_TRUNC(usage_start_time, DAY) = "{curr_utc.strftime('%Y-%m-%d')}")
    GROUP BY project_name, sku_id, sku_def, service_id, service_def
    ORDER BY project_name;"""
)
query_job = client.query(QUERY)  # API request
rows = query_job.result()

/Users/ejcv/miniconda3/envs/test_alerts/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/ejcv/miniconda3/envs/test_alerts/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. F

In [7]:
rows.to_dataframe()

,project_name,sku_id,sku_def,service_id,service_def,prev_day,curr_day
0,rax-datascience,C024-9C10-2A5B,CPU Time,29E7-DA93-CA13,Cloud Functions,0.000028,0.000068
1,rax-datascience,AB59-C5C0-8570,BigQuery Storage API - Read Bytes Scanned (US),DCC9-8DB9-673F,BigQuery Storage API,0.000000,0.000000
2,rax-datascience,A464-9020-6404,Build time,8B5D-EF7D-EB12,Cloud Build,0.000000,0.000000
3,rax-datascience,8502-299A-ABAF,Artifact Registry Storage,149C-F9EC-3994,Artifact Registry,0.137703,0.137708
4,rax-datascience,22EB-AAE8-FBCD,Download Worldwide Destinations (excluding Asi...,95FF-2EF5-5EA1,Cloud Storage,0.000002,0.000000
5,rax-datascience,7756-ADEF-84F4,Secret version replica storage,EE82-7A5E-871C,Secret Manager,0.032013,0.031983
6,rax-datascience,8E10-82EB-6917,Invocations,29E7-DA93-CA13,Cloud Functions,0.000000,0.000000
7,rax-datascience,B068-6B82-C017,Network Egress from us-central1,29E7-DA93-CA13,Cloud Functions,0.000000,0.000000
8,rax-datascience,F01C-3EA0-06CD,Memory Time,29E7-DA93-CA13,Cloud Functions,0.000004,0.000009
9,rax-datascience,8AE7-5BBD-8F38,Multi-Region Standard Class B Operations,95FF-2EF5-5EA1,Cloud Storage,0.000050,0.000004


## This query is to get the monthly consumption

In [19]:
now = datetime.now(timezone.utc)
prev_utc = now -relativedelta(months=+1)
curr_utc = now

QUERY = (
    f"""SELECT
    project.name as project_name,
    sku.id as sku_id,
    sku.description as sku_def,
    service.id as service_id,
    service.description as service_def,
    SUM(CASE WHEN EXTRACT(MONTH FROM usage_start_time) = {prev_utc.month} THEN cost ELSE 0 END) AS prev_month,
    SUM(CASE WHEN EXTRACT(MONTH FROM usage_start_time) = {curr_utc.month} THEN cost ELSE 0 END) AS curr_month,
    FROM `{BQ_TABLE}`
    WHERE project.name = 'rax-datascience' 
    AND (DATE_TRUNC(usage_start_time, MONTH) = "{prev_utc.strftime('%Y-%m-01')}" or DATE_TRUNC(usage_start_time, MONTH) = "{curr_utc.strftime('%Y-%m-01')}")
    GROUP BY project_name, sku_id, sku_def, service_id, service_def
    ORDER BY project_name;"""
)
query_job = client.query(QUERY)  # API request
rows = query_job.result()

## This query is to get the weekly consumption

In [65]:
now = datetime.now(timezone.utc).isocalendar()[1]
prev_week = now - 1
curr_week = now

QUERY = (
    f"""SELECT
    project.name as project_name,
    sku.id as sku_id,
    sku.description as sku_def,
    service.id as service_id,
    service.description as service_def,
    SUM(CASE WHEN EXTRACT(ISOWEEK FROM usage_start_time) = {prev_week} THEN cost ELSE 0 END) AS prev_week,
    SUM(CASE WHEN EXTRACT(ISOWEEK FROM usage_start_time) = {curr_week} THEN cost ELSE 0 END) AS curr_week,
    FROM `{BQ_TABLE}`
    WHERE project.name = 'rax-datascience' 
    AND (EXTRACT(ISOWEEK FROM usage_start_time) = {prev_week} or EXTRACT(ISOWEEK FROM usage_start_time) = {curr_week})
    GROUP BY project_name, sku_id, sku_def, service_id, service_def
    ORDER BY project_name;"""
)
query_job = client.query(QUERY)  # API request
rows = query_job.result()

In [66]:
prev_week

6

In [67]:
datetime.now(timezone.utc).isocalendar()

datetime.IsoCalendarDate(year=2022, week=7, weekday=5)

In [68]:
df = rows.to_dataframe()

In [69]:
df

,project_name,sku_id,sku_def,service_id,service_def,prev_week,curr_week
0,rax-datascience,E5F0-6A5D-7BAD,Standard Storage US Regional,95FF-2EF5-5EA1,Cloud Storage,0.000000,0.000000
1,rax-datascience,FED8-23FC-20EF,Analysis Slots Attribution,24E6-581D-38E5,BigQuery,0.000000,0.000000
2,rax-datascience,947D-3B46-7781,Active Storage,24E6-581D-38E5,BigQuery,1.838560,0.958063
3,rax-datascience,143F-A1B0-E0BE,Log Volume,5490-F7B7-8DF6,Cloud Logging,0.000000,0.000000
4,rax-datascience,0D5D-6E23-4250,Standard Storage US Multi-region,95FF-2EF5-5EA1,Cloud Storage,0.094591,0.055638
5,rax-datascience,7870-010B-2763,Regional Standard Class B Operations,95FF-2EF5-5EA1,Cloud Storage,0.000000,0.000022
6,rax-datascience,6FE6-D064-82C6,BigQuery Flat Rate Flex for US (multi-region),16B8-3DDA-9F10,BigQuery Reservation API,2.077777,0.000000
7,rax-datascience,22EB-AAE8-FBCD,Download Worldwide Destinations (excluding Asi...,95FF-2EF5-5EA1,Cloud Storage,0.000641,0.000002
8,rax-datascience,1DF5-1F98-1DD1,Analysis,24E6-581D-38E5,BigQuery,26.693103,8.084075
9,rax-datascience,A5D6-43A2-28A7,Jobs,1F14-4801-0E16,Cloud Scheduler,0.000000,0.002764


In [75]:
import pymsteams
import random

In [62]:
myTeamsMessage = pymsteams.ConnectorCard("https://raxglobal.webhook.office.com/webhookb2/958f9af0-e01e-4dd7-a71d-9d6e8f85e1e3@570057f4-73ef-41c8-bcbb-08db2fc15c2b/IncomingWebhook/ba01367e4b864dcbaa9b3df47f354687/03c80c6a-e55a-4acf-ab93-0ed10e8993aa")
myTeamsMessage.text("Hola bola")
myTeamsMessage.send()

True

In [71]:
with open("quotes.txt") as f:
    quotes = f.read().splitlines()

In [76]:
random.choice(quotes)

"“Why should I want to make anything up? Life's bad enough as it is without wanting to invent any more of it.” "

In [79]:

myTeamsMessage.text(f"Nothing to alert today \n \t {random.choice(quotes)}")
myTeamsMessage.send()

True

In [81]:
not []

True